In [79]:
import pandas
import numpy
from functools import reduce

pandas.options.mode.chained_assignment = None

data = {
    "circ": pandas.read_csv("input/Circulatory Mortality.csv", index_col=False),
    "resp": pandas.read_csv("input/Respiratory Mortality.csv", index_col=False),
    "all": pandas.read_csv("input/All Cause Mortality.csv", index_col=False),
    "circ_est": pandas.read_csv(
        "input/Circulatory Mortality with estimates.csv", index_col=False
    ),
    "resp_est": pandas.read_csv(
        "input/Respiratory Mortality with estimates.csv", index_col=False
    ),
    "all_est": pandas.read_csv(
        "input/All Cause Mortality with estimates.csv", index_col=False
    ),
}


def convert_ids_to_int(df):
    df = df[df["State ANSI"].notnull()]
    df["STATEFP"] = pandas.to_numeric(df["State ANSI"], downcast="integer")
    df["COUNTYFP"] = pandas.to_numeric(df["County ANSI"], downcast="integer")
    return df.drop(columns=["State ANSI", "County ANSI"])


def calculate_mortality(df):
    df["mort_0-5"] = df["deaths_0-5"] / df["Population_0-5"].replace({0: numpy.nan})
    df["mort_5-25"] = df["deaths_5-25"] / df["Population_5-25"].replace({0: numpy.nan})
    df["mort_25+"] = df["deaths_25+"] / df["Population_25+"].replace({0: numpy.nan})
    return df


def remove_percent_deaths(df):
    if (
        "Percent Deaths 0-5" in df
        and "Percent Deaths 5-25" in df
        and "Percent Deaths 25+" in df
    ):
        return df.drop(
            columns=["Percent Deaths 0-5", "Percent Deaths 5-25", "Percent Deaths 25+"]
        )
    else:
        return df

def remove_zeros(df):
    return df


def append_suffix(df, suffix):
    return df.rename(
        columns={
            "deaths_0-5": "deaths_0-5_" + suffix,
            "deaths_25+": "deaths_25+_" + suffix,
            "deaths_5-25": "deaths_5-25_" + suffix,
            "mort_0-5": "mort_0-5_" + suffix,
            "mort_5-25": "mort_5-25_" + suffix,
            "mort_25+": "mort_25+_" + suffix,
        }
    )


rename_map = {
    "Deaths_0-5": "deaths_0-5",
    "Deaths_25+": "deaths_25+",
    "Deaths_5-25": "deaths_5-25",
}


def clean(df, key):
    df = convert_ids_to_int(df)
    df = df.rename(columns=rename_map)
    df = calculate_mortality(df)
    df = remove_percent_deaths(df)
    df = append_suffix(df, key)
    df = df.replace({0: numpy.nan})
    df = df.round(6)
    return df


data = {key: clean(df, key) for key, df in data.items()}

data_merged = reduce(
    lambda left, right: pandas.merge(
        left,
        right,
        on=[
            "STATEFP",
            "COUNTYFP",
            "Population_0-5",
            "Population_25+",
            "Population_5-25",
        ],
    ),
    data.values(),
)

data_merged.to_csv("output/mortality.csv", index=None)
data_merged


TypeError: to_csv() got an unexpected keyword argument 'round'